In [1]:
# ============ 改进版算法特征工程 ============

def create_enhanced_algorithm_features(self, df):
    """创建增强版算法特征 - 添加更多专业算法"""
    print("🚀 创建增强版算法特征...")
    
    feature_df = df.copy()
    cpu_col = 'cpu_utilization'
    algorithm_features = []
    
    # 准备数据
    cpu_data = feature_df[cpu_col].values.reshape(-1, 1)
    cpu_series = feature_df[cpu_col]
    
    # ============ 现有算法 ============
    
    # 1. Isolation Forest
    try:
        iso_forest = IsolationForest(contamination=0.08, random_state=42)
        iso_scores = iso_forest.fit_predict(cpu_data)
        feature_df['iso_forest_anomaly'] = (iso_scores == -1).astype(int)
        algorithm_features.append('iso_forest_anomaly')
    except Exception as e:
        print(f"   ⚠️ Isolation Forest失败: {e}")
        feature_df['iso_forest_anomaly'] = 0
    
    # 2. Local Outlier Factor
    try:
        lof = LocalOutlierFactor(n_neighbors=min(20, len(feature_df)-1), contamination=0.08)
        lof_scores = lof.fit_predict(cpu_data)
        feature_df['lof_anomaly'] = (lof_scores == -1).astype(int)
        algorithm_features.append('lof_anomaly')
    except Exception as e:
        print(f"   ⚠️ LOF失败: {e}")
        feature_df['lof_anomaly'] = 0
    
    # 3. Z-score (滚动窗口)
    rolling_mean = cpu_series.rolling('60min').mean()
    rolling_std = cpu_series.rolling('60min').std()
    feature_df['cpu_zscore'] = (cpu_series - rolling_mean) / (rolling_std + 1e-8)
    feature_df['zscore_anomaly'] = (np.abs(feature_df['cpu_zscore']) > 3).astype(int)
    
    # 4. IQR方法
    rolling_q75 = cpu_series.rolling('60min').quantile(0.75)
    rolling_q25 = cpu_series.rolling('60min').quantile(0.25)
    rolling_iqr = rolling_q75 - rolling_q25
    iqr_upper = rolling_q75 + 1.5 * rolling_iqr
    iqr_lower = rolling_q25 - 1.5 * rolling_iqr
    feature_df['iqr_anomaly'] = ((cpu_series > iqr_upper) | (cpu_series < iqr_lower)).astype(int)
    
    algorithm_features.extend(['cpu_zscore', 'zscore_anomaly', 'iqr_anomaly'])
    
    # ============ 新增专业算法 ============
    
    # 5. EWMA (指数加权移动平均) - 您提到的算法
    print("   📈 添加EWMA异常检测...")
    alpha = 0.3  # 平滑参数
    ewma_mean = cpu_series.ewm(alpha=alpha).mean()
    ewma_std = cpu_series.ewm(alpha=alpha).std()
    feature_df['ewma_score'] = np.abs(cpu_series - ewma_mean) / (ewma_std + 1e-8)
    feature_df['ewma_anomaly'] = (feature_df['ewma_score'] > 3).astype(int)
    algorithm_features.extend(['ewma_score', 'ewma_anomaly'])
    
    # 6. LSTM Autoencoder模拟 (简化版 - 使用滑动窗口重构误差)
    print("   🧠 添加重构误差检测...")
    window_size = 10
    reconstruction_errors = []
    for i in range(len(cpu_series)):
        if i < window_size:
            reconstruction_errors.append(0)
        else:
            # 使用前N个点的均值预测当前点
            window_data = cpu_series.iloc[i-window_size:i]
            predicted = window_data.mean()
            error = abs(cpu_series.iloc[i] - predicted)
            reconstruction_errors.append(error)
    
    feature_df['reconstruction_error'] = reconstruction_errors
    # 使用重构误差的95分位数作为阈值
    threshold = np.percentile(reconstruction_errors, 95)
    feature_df['reconstruction_anomaly'] = (feature_df['reconstruction_error'] > threshold).astype(int)
    algorithm_features.extend(['reconstruction_error', 'reconstruction_anomaly'])
    
    # 7. 变化点检测 (Change Point Detection)
    print("   📊 添加变化点检测...")
    # 计算滑动方差比值来检测结构性变化
    short_window = 15
    long_window = 60
    short_var = cpu_series.rolling(f'{short_window}min').var()
    long_var = cpu_series.rolling(f'{long_window}min').var()
    variance_ratio = short_var / (long_var + 1e-8)
    feature_df['variance_ratio'] = variance_ratio
    feature_df['changepoint_anomaly'] = (variance_ratio > 2.0).astype(int)  # 短期方差明显高于长期
    algorithm_features.extend(['variance_ratio', 'changepoint_anomaly'])
    
    # 8. 季节性分解异常检测
    print("   🔄 添加季节性异常检测...")
    # 简化的季节性分解：每小时模式
    hourly_pattern = cpu_series.groupby(cpu_series.index.hour).transform('mean')
    seasonal_residual = cpu_series - hourly_pattern
    seasonal_std = seasonal_residual.rolling('120min').std()
    feature_df['seasonal_residual'] = seasonal_residual
    feature_df['seasonal_anomaly'] = (np.abs(seasonal_residual) > 2 * seasonal_std).astype(int)
    algorithm_features.extend(['seasonal_residual', 'seasonal_anomaly'])
    
    # ============ 增强的集成特征 ============
    
    # 扩展算法列表
    enhanced_algorithm_cols = [
        'iso_forest_anomaly', 'lof_anomaly', 'zscore_anomaly', 'iqr_anomaly',
        'ewma_anomaly', 'reconstruction_anomaly', 'changepoint_anomaly', 'seasonal_anomaly'
    ]
    
    # 重新计算集成特征
    feature_df['algorithm_consensus'] = feature_df[enhanced_algorithm_cols].sum(axis=1)
    feature_df['algorithm_agreement'] = (feature_df['algorithm_consensus'] >= 3).astype(int)  # 至少3个算法一致
    feature_df['strong_consensus'] = (feature_df['algorithm_consensus'] >= 5).astype(int)     # 强一致性：5个以上
    
    # 加权一致性（给不同算法不同权重）
    weights = {
        'iso_forest_anomaly': 0.15,     # 机器学习算法权重高
        'lof_anomaly': 0.15,
        'ewma_anomaly': 0.15,           # 时序专用算法权重高
        'seasonal_anomaly': 0.15,
        'zscore_anomaly': 0.1,          # 统计算法权重中等
        'iqr_anomaly': 0.1,
        'reconstruction_anomaly': 0.1,
        'changepoint_anomaly': 0.1
    }
    
    weighted_score = sum(feature_df[col] * weight for col, weight in weights.items())
    feature_df['weighted_consensus'] = weighted_score
    feature_df['weighted_anomaly'] = (weighted_score > 0.3).astype(int)  # 加权阈值
    
    algorithm_features.extend([
        'algorithm_consensus', 'algorithm_agreement', 'strong_consensus',
        'weighted_consensus', 'weighted_anomaly'
    ])
    
    print(f"   ✅ 增强算法特征: {len(algorithm_features)} 个")
    print(f"   📊 算法覆盖:")
    print(f"      - 机器学习: Isolation Forest, LOF")
    print(f"      - 时序专用: EWMA, 季节性分解")
    print(f"      - 统计方法: Z-score, IQR")
    print(f"      - 结构检测: 变化点, 重构误差")
    print(f"      - 集成方法: 投票, 加权一致性")
    
    return feature_df, algorithm_features

# 为CPUFeatureEngineer类添加这个新方法
CPUFeatureEngineer.create_enhanced_algorithm_features = create_enhanced_algorithm_features

print("✅ 增强版算法特征函数已添加!")

NameError: name 'CPUFeatureEngineer' is not defined